In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### AWS DynamoDB Commmand Line (Cli)
#### 1. Create Tables
```
aws dynamodb create-table --table-name user_notes_test --attribute-definitions AttributeName=user_id,AttributeType=S AttributeName=timestamp,AttributeType=N --key-schema AttributeName=user_id,KeyType=HASH AttributeName=timestamp,KeyType=RANGE --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1
```
#### 2. Describe Tables
```aws dynamodb describe-table --table-name user_notes_test```
#### 3. List Tables
```aws dynamodb list-tables```
#### 4. Delete Tables
```aws dynamodb delete-table --table-name td_notes_test```
#### 5. Add Items to Tables
```aws dynamodb put-item --table-name user_notes_test --item file://item.json```
#### 6. Update items to Tables
```
aws dynamodb update-item --table-name user_notes_test --key file://key.json --update-expression "SET #t = :t" --expression-attribute-names file://attribute-names.json --expression-attribute-values file://attribute-values.json
```

The key.json:
```
{
    "user_id": { "S": "1234abcd" },
    "timestamp": { "N": "1561922330" }
}
```

The attribute-names.json:
```
{
    "#t": "title"
}
```

The attribute-values.json:
```
{
    ":t": { "S": "My updated title" }
}
```
#### 7. Delete Items
define the keys of the item to be deleted in the aws cli, as the following:
```
aws dynamodb delete-item --table-name user_notes_test --key file://key.json
```

#### 8. batch Loading and Deleting Items
1. prepare the items file that contains all the items to be added, and the primary keys of the items to be deleted, as the following example:
```
{
    "user_notes_test": [{
            "PutRequest": {
                "Item": {
                    "user_id": { "S": "1111" },
                    "timestamp": { "N": "1525013088" },
                    "content": { "S": "Content 1111" },
                    "title": { "S": "Title 1111" }
                }
            }
        },
        {
            "PutRequest": {
                "Item": {
                    "user_id": { "S": "2222" },
                    "timestamp": { "N": "1525013088" },
                    "content": { "S": "Content 2222" },
                    "title": { "S": "Title 2222" }
                }
            }
        },
        {
            "DeleteRequest": {
                "Key": {
                    "user_id": { "S": "1234abcd" },
                    "timestamp": { "N": "1525013000" }
                }
            }
        }
    ]
}
```
2. run the following command:   
```aws dynamodb batch-write-item --request-items file://items.json```   
notice that the table name is not necessary, since the table name(s) are defined in the item.json file. Therefore, it is possible to operate on multiple tables using the batch-write_item command, and we only need to define the table names and the corresponding item attributes and keys in the json file.

#### 9. Read Items from Tables
```aws dynamodb get-item --table-name user_notes_test --key file://read-key.json```   
the read-key.json is in the following format:
```
{    
    "user_id": { "S": "B" },    
    "timestamp": { "N": "2" }    
}
```

#### 10. Query
```
aws dynamodb query --table-name user_notes_test --key-condition-expression "user_id =:uid AND #t > :t" --expression-attribute-value file://expression-attribute-values.json --expression-attribute-name file://expression-attribute-name.json --filter-expression "cat =:cat"
```
The expression-attribute-values.json file is in the following format:
```
{   
    ":uid": { "S": "A" },   
    ":t":{"N": "5"},   
    ":cat":{"S":"todo"}   
}
```

The expression-attribute-name.json file is in the following format:
```
{      
    "#t":"timestamp"      
 }
 ```

Notice that the filter happens after the query. That is why we may have ScannedCount > Count in the final output.

If we ask to return the consumed capacity, it will give us the consumed capacity in RCU. The eventually and strongly consistency will give us different consumed-capacity. We can run these command as the following:
```
aws dynamodb query --table-name user_notes_test --key-condition-expression "user_id =:uid AND #t >:t" --expression-attribute-value file://expression-attribute-values.json --expression-attribute-name file://expression-attribute-name.json --filter-expression "cat =:cat" --return-consumed-capacity INDEXES [--consistent-read    
  (strong consistentcy)]  --scan-index-forward (default, ascending in sort key)/ no-scan-index-forward (descending in sort key)
```  
  
#### 11. Scan
Search the entire table across multiple partition (each scan retrun 1 MB data at maximum) 
```aws dynamodb scan --table-name user_notes_test --filter-expression "username =:uname" --expression-attribute-values file://uname.json
```

The uname.json has the following format:
```
{
    ":uname": { "S": "User B" }
}
```
